In [1]:
import argparse
import numpy as np
from itertools import count
import time
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
d1 = torch.device("cuda:0")
d2 = torch.device("cuda:1")

In [2]:
# input_test = torch.randn(80, 1, dim).float().cuda()
class Net(nn.Module):
    def __init__(self, i=1):
        super(Net, self).__init__()
        i = math.ceil(i)
        if not i >= 1:
            i = 1
        self.l1 = nn.Linear(i, i)
        
    
    def forward(self, x):
        y = self.l1(x)
        return y

In [ ]:
nets = []
i = 0
size = 500
while True:
    i += 1
    size = size + 1
    try:
        net = Net(size).to(d1)
        nets.append(net)
    except:
        size /= 2
        time.sleep(0.5)
    try:
        net = Net(size).to(d2)
        nets.append(net)
    except:
        size /= 2
        time.sleep(0.5)
    print(size)

501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
